<h1><center>DecisionTree - Regression</center></h1>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
import warnings
warnings.filterwarnings('ignore')

import random
random.seed(10)

In [6]:
hitters_data = pd.read_csv("../../Data/hitters_clean.csv")

In [7]:
hitters_data.head()

,Name,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,Salary
0,-Alan Ashby,315,81,7,24,38,39,14,3449,835,69,321,414,375,632,43,10,475.0
1,-Alvin Davis,479,130,18,66,72,76,3,1624,457,63,224,266,263,880,82,14,480.0
2,-Andre Dawson,496,141,20,65,78,37,11,5628,1575,225,828,838,354,200,11,3,500.0
3,-Andres Galarraga,321,87,10,39,42,30,2,396,101,12,48,46,33,805,40,4,91.5
4,-Alfredo Griffin,594,169,4,74,51,35,11,4408,1133,19,501,336,194,282,421,25,750.0


### Predicting Baseball Players’ Salaries Using Regression Trees. For simplicity we will take  'Hits', 'Years' features and predict 'Salary' of a baseball player.

In [8]:
hitters_data[['Hits', 'Years', 'Salary']].head()

,Hits,Years,Salary
0,81,14,475.0
1,130,3,480.0
2,141,11,500.0
3,87,2,91.5
4,169,11,750.0


In [9]:
hitters_data[['Hits', 'Years', 'Salary']].describe()

,Hits,Years,Salary
count,263.000000,263.000000,263.000000
mean,107.828897,7.311787,535.925882
std,45.125326,4.793616,451.118681
min,1.000000,1.000000,67.500000
25%,71.500000,4.000000,190.000000
50%,103.000000,6.000000,425.000000
75%,141.500000,10.000000,750.000000
max,238.000000,24.000000,2460.000000


In [12]:
hitters_data[hitters_data.Years < 4.5]['Salary'].mean()

225.83147777777776

In [7]:
hitters_data[(hitters_data.Years >= 4.5) & (hitters_data.Hits < 117.5)]['Salary'].mean()

464.9166777777778

In [8]:
hitters_data[(hitters_data.Years >= 4.5) & (hitters_data.Hits >= 117.5)]['Salary'].mean()

949.1707590361445

### We can represent a simple tree model (for interpretation purpose) as shown below.
<img height="400" width="550" src="../img/regression/sample_tree.png">

    
  * **The model will predict salary of player based on below conditions.**
    * If "years of experience < 4.5, Predicted Salary = **average salaries of players with "years of experience < 4.5". **
    * If expreience >= 4.5 years then
      * If Hits < 117.5, Predicted Salary = **average salaries of players with "years of experience >= 4.5" AND "Hits < 117.5".**
      * If Hits >= 117.5, Predicted Salary = **average salaries of players with "years of experience >= 4.5" AND "Hits >= 117.5".**
      
### The algorithm consists of a series of splitting rules, starting at the top of the tree. One can interpret these splitting rules as shown below (Denoted with R1, R2 and R3).
<img height="400" width="400" src="./img/regression/sample_tree_regions.png">

  * These three regions can be written as (the regions R 1 , R 2 , and R 3 are known as **terminal nodes** or **leaves** of the tree) 
    * R1 ={X | Years<4.5}
    * R2 ={X | Years>=4.5, Hits<117.5}
    * R3 ={X | Years>=4.5, Hits>=117.5}
    
  * The points along the tree where the predictor space is split are referred to as **internal nodes**.(the two internal
nodes are "Years<4.5" and "Hits<117.5")

## We now discuss the process of building a regression tree. Roughly speaking, there are two steps.
  1. Algorithm divide the predictor space - that is, the set of possible values (samples) for $ X_1 , X_2 , . . . , X_p $ —into J distinct and non-overlapping regions, $ R_1 , R_2 , . . . , R_J $
  2. For every observation that falls into the region $ R_j $, Algorithm make the same prediction, which is simply the mean of the response values for the training observations in $ R_j $ .
    * If we take above example we have three regions, $ R_1, R_2, R_3 $. All the samples x fall in $ R_1 $ will be predicted as 225.83 K salary, in $ R_2 $ will be predicted as 465.92 K salary, in $ R_3 $ will be predicted as 949.17 K salary,.  
    
### How to select the best predictor to split, and the cut point (the value of 4.5 in years < 4.5) for the predictor -  at each node  ?
  * Algorithm takes a **top-down, greedy** approach that is known as **recursive binary splitting**.
  * The approach is **top-down** because it begins at the top of the tree (at which point all observations belong to a single region) and then successively splits the predictor space; each split is indicated via two new branches further down on the tree.
  * It is **greedy** because at each step of the tree-building process, the best split is made at that particular step, rather than looking ahead and picking a split that will lead to a better tree in some future step.

### Recursive Binary Split: 
  * The goal is to find boxes $ R_1, R_2, . . . , R_J $ that minimize the RSS, given by 
  $$ RSS = \sum_{j=1}^J \sum_{i \space \epsilon \space R_j} {(y_i - y_{R_j})}^2 \space , $$
    * Where **$ y_{R_j}^| $ is the mean response for the training observations within the $ j_{th} $ box**.
  * We consider all predictors $ X_1, X_2, . . . ,X_p $, and all possible values of the cutpoint "s" for each of the predictors, and then choose the predictor and cutpoint such that the resulting tree has the lowest RSS.
  $$ R_1 (j,s) = \{ X \space | \space X_j < s \} \space and \space R_2 (j,s) = \{ X \space | \space X_j >= s \}, $$
  and we **seek** the value of **j** and **s** that minimize the equation
  $$ \sum_{i \space :  \space x_i \space \epsilon \space R_1 (j,s)} (y_i - y_{R_1}^|)^2 \space + \space \sum_{i \space :  \space x_i \space \epsilon \space R_2 (j,s)} (y_i - y_{R_2}^|)^2 \space, $$
  * We repeat the process, looking for the best predictor and best cutpoint in order to split the data further so as to minimize the RSS within each of the resulting regions. However, this time, instead of splitting the entire predictor space, we split one of the two previously identified regions.We now have three regions. Again, we look to split one of these three regions further, so as to minimize the RSS. The process continues until a stopping criterion is reached; for instance, we may continue until no region contains more than five observations.


## Decision Tree Regression

In [9]:
from sklearn.tree import DecisionTreeRegressor

In [10]:
housing_train_set = pd.read_csv('../../data/cleaned_housing_train_set.csv')
X_features  = [feature for feature in housing_train_set.columns if feature not in ['SalePrice', 'Id']]
y_feature = 'SalePrice'

In [11]:
class InputOutputFeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, in_attr_list, our_feature):
        self.in_attr_list = in_attr_list
        self.our_feature = our_feature
        #self.mean = None
        #self.std = None
    def fit(self, X, y=None):
        #self.mean = X['feature'].mean()
        #self.std = X['feature'].mean()
        return self
    def transform(self, X):
        #X['feature'] = X['feature'] - self.mean
        #X['feature'] = X['feature'] /self.std
        return X[self.in_attr_list].values, X[self.our_feature].values

In [12]:
def defineInOutPipeling(X_feat, y_feat):
    pipeline_inout_features = Pipeline([
        ('inputOutputFeatureSelector', InputOutputFeatureSelector(X_feat, y_feat))
    ])
    return pipeline_inout_features

In [13]:
pipeline_inout_features = defineInOutPipeling(X_features, y_feature)

In [14]:
X_train, y_train = pipeline_inout_features.fit_transform(housing_train_set)

In [15]:
tree_reg = DecisionTreeRegressor(max_depth=3, random_state=32)
tree_reg.fit(X_train, y_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=3,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=32, splitter='best')

In [16]:
y_train_pred =tree_reg.predict(X_train)

In [17]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
def printREGAccuracyScores(y, pred_y):
    mse =  mean_squared_error(y, pred_y)
    print("Root Mean Squared Error : ", np.sqrt(mse))
    r2 = r2_score(y, pred_y)
    print("R-Squared Error : ", r2)

In [18]:
printREGAccuracyScores(y_train, y_train_pred)

Root Mean Squared Error :  41635.14407201488
R-Squared Error :  0.743141538018619


### The above scores doesn't look good, lets try "RandomizedSearchCV" to find best parameters.

In [18]:
import scipy
from scipy.stats import randint
import numpy as np
ri = randint(low=1, high=200)
print(ri)

In [19]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {
        'max_leaf_nodes': randint(low=1, high=200),
        'max_depth': randint(low=1, high=100),
        'min_samples_leaf' : randint(low=1, high=100)
    }
grd_tree_reg = DecisionTreeRegressor(random_state=32)
rnd_tree_search = RandomizedSearchCV(grd_tree_reg, param_distributions=param_distribs,
                                n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=32)
rnd_tree_search.fit(X_train, y_train)

RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=DecisionTreeRegressor(ccp_alpha=0.0,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features=None,
                                                   max_leaf_nodes=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   presort='deprecated',
                                                   random_state=32,
                                                   splitter='best'),
                   iid='dep

In [20]:
rnd_tree_search.best_params_

{'max_depth': 88, 'max_leaf_nodes': 44, 'min_samples_leaf': 6}

In [21]:
rnd_tree_search.best_estimator_

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=88,
                      max_features=None, max_leaf_nodes=44,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=6, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=32, splitter='best')

In [22]:
best_tree_reg = rnd_tree_search.best_estimator_
best_tree_reg.fit(X_train, y_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=88,
                      max_features=None, max_leaf_nodes=44,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=6, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=32, splitter='best')

In [23]:
best_y_train_pred = best_tree_reg.predict(X_train)

In [24]:
printREGAccuracyScores(y_train, best_y_train_pred)

Root Mean Squared Error :  28619.646651250918
R-Squared Error :  0.8786325382974469


In [25]:
housing_test_set= pd.read_csv('../../data/cleaned_housing_test_set.csv')
X_test, y_test = pipeline_inout_features.transform(housing_test_set)

In [26]:
best_y_test_pred = best_tree_reg.predict(X_test)

In [27]:
printREGAccuracyScores(y_test, best_y_test_pred)

Root Mean Squared Error :  35018.4293086764
R-Squared Error :  0.7343207976646908
